In [1]:
# Libraries to access external data
# PyVO provides generic access to VO protocols (ex: TAP)
import pyvo
# astroquery modules are specific to astronomy services, there are not necessarily VO,
# but more and more do switch to VO protocols these days
# here we import the MOCServer and Vizier modules
from astroquery.mocserver import MOCServer 
from astroquery.vizier import Vizier

# The mocpy library is an astropy-affiliated library
# It allows to manipulate MOCs
from mocpy import STMOC, MOC, TimeMOC, WCS

# General astropy utilitary methods and classes
from astropy.coordinates import Longitude, Latitude, Angle, SkyCoord
from astropy.time import Time, TimeDelta
import astropy.units as u

# For vizaulisation
import matplotlib.pyplot as plt
from ipyaladin import Aladin


# Accessing data through space and time criteria

## This notebook focuses on the use of the Multi-Order Coverage (MOC) and Space-Time MOC (STMOC) for querying and analyzing astronomical data. 

### Key words:

**Hierarchical Progressive Survey (HiPS)**: HiPS describes a generic method for packaging, storing, querying, and describing astronomical data. It stores data in a folder structure that mimics the [HEALPix tesselation of the sky](https://healpix.jpl.nasa.gov/pdf/intro.pdf).

**Multi-Order Coverage (MOC)**: MOC is a data structure used to describe arbitrary regions of the sky with variable resolution. It is particularly useful for handling large-scale astronomical surveys and for efficiently querying regions of interest.

**Space-Time MOC (STMOC)**: STMOC extends the concept of MOC to include the temporal dimension, allowing for the representation of regions of the sky that change over time. This is useful for time-domain astronomy, where the visibility of sources varies.

### Scientific Motivation:

Gamma-Ray Bursts (GRBs) are among the most energetic and mysterious phenomena in the universe. These brief but intense flashes of gamma rays are believed to originate from cataclysmic events such as the collapse of massive stars or the merger of neutron stars. Observing GRBs across multiple wavelengths and with different telescopes provides crucial insights into their nature and origins. In this study, we focus on GRBs observed by the Fermi Gamma-ray Space Telescope and the XMM-Newton X-ray Observatory, and demonstrate how the use of Space-Time Multi-Order Coverage (STMOC) can help identify and analyze these fascinating events.

## The HEALPix grid and its link to HiPS surveys

HiPS is a way of storing data sets (it can be catalogs, or images for example) that follows the HEALPix tesselation of the sky. This is particularly optimized for vizualisation.

![HiPS_and_the_HEALPix_grid](HiPS_and_the_HEALPix_grid.png)

Figure 1: Standards to represent the sky, or parts of the sky. (a) The HEALPix grid (b) The HiPS standard (c) Visualisation in ipyaladin (d), and a MOC.

### The HEALPix grid

With the HEALPix grid, the sphere is mapped by equal-area tiles. In figure 1a, we see an orthographic representation of the 12 tiles at order zero and of the 768 tiles at order 3. This is extracted from [Gorski *et.al.*](https://iopscience.iop.org/article/10.1086/427976/pdf).

`mocpy` is a python library that allows to manipulate MOCs. MOCs are made of HEALPix cells, so mocpy has a few methods that allow to get information about the HEALPix tesselation.

In [ ]:
# we can check the number of cells at order 0 in the HEALPix tesselation
MOC.n_cells(0)

In [ ]:
# or at an other order
MOC.n_cells(3)

In [ ]:
# we could also have calculated the order 3 by knowing that each cell is subdivised in four parts at the next order
n_cells_order3 = MOC.n_cells(0) * 4**3
n_cells_order3

Let's vizualize the HEALPix grid in the ipyaladin widget, see Figure1c to see where to find the parameter. If you zoom in and out and move like crazy (or if you have a slower internet connexion), you'll see the images that appear as they are downloaded by ipyaladin. They have the shape of HEALPix cells. The more you zoom in, the more precise (meaning higher resolution) the printed image will be.

In [2]:
aladin = Aladin()
aladin

Aladin(init_options=['_fov', '_target', 'background_color', 'coo_frame', 'full_screen', 'grid_color', 'grid_op…

We can diplucate this view by right clicking on the cell and selecting `Create new view for cell output`. The view can then be dragged anywhere you prefer. This allows to always have an eye on the widget while exploring the data.

### The HiPS standard

The images displayed by Aladin follow the HiPS standard, in which a folder corresponds to a HEALPix order, see figure1b. In figure 1b, the example images are from a SDO/AIA observation of the Sun. At all orders, the images have the same number of pixels (here 512*512), but they represent a different sky fraction. We see that the pixel 0 at order 3 contains the pixel 0 and 1 of the more precise order.

### Finding HiPS surveys

More and more astronomical data centers start to offer HiPS surveys for sky vizualizations.

One way of finding HiPS surveys is to use `ipyaladin`'s interface, see Figure2, and follow the steps to get the two MeerKAT HiPS.

![findind_HiPS](finding_hips_with_ipyaladin.png)


In [6]:
aladin.target = "Galactic Center"

ValueError: Invalid character at col 0 parsing angle 'Center'

This calls one of the less known services of the CDS, the MOCServer. The search bar only works if you already know more or less the name of the survey. For more control on search parameters, we will use the module `mocserver` from the `astroquery` library to access the MOCServer.

Here we query the MOC server to find HiPS datasets related to Fermi observations. Fermi cameras are sensitive to sources that emit in gamma-rays. GRBs are the ideal candidate for Fermi.

In [4]:
fermi_hips = MOCServer.find_datasets(meta_data="obs_description=*Fermi*&&hips_*=*", casesensitive=False)

In [5]:
len(fermi_hips)

9

We found nine HiPS surveys with the word `Fermi` in their descriptions. The object that we got from astroquery is a table containing information about the HiPS surveys.

In [ ]:
fermi_hips["ID"]

## 
GRBs can be studied in conjunction with other instruments spanning wide range of wavelengths in order to understand the various propoerties of the GRB. The bursts of gamma-rays produced b the GRBs, collide with each other and also the interstellar medium and produce what we call the "after glow". The "after glow" contains electromagnetic radiaitons ranging from X-rays to radio. Let's explore the GRBs also detected in X-rays. As shown above, we can obtain HiPs for the XMM-Newton Telescope. XMM-Newton is a powerful telescope that observes the X-ray Universe. 

In [ ]:
xmm_hips = MOCServer.find_datasets(meta_data="obs_description=*xmm*&&hips_*=*", casesensitive=False)
xmm_hips

In [ ]:
xmm_hips["ID"]

Now in aladin

## Aladin

We can integrate the Aladin Lite sky atlas into a Jupyter Notebook. It aids in visualisation and exploration of of astronomical images and data. 

In [ ]:
aladin = Aladin(survey="CDS/P/Fermi/color", height=600)
aladin

In [ ]:
aladin.coo_frame = "galactic"

In [ ]:
#aladin.overlay_survey = "CDS/P/MeerKAT/Galactic-Centre-1284MHz-StokesI"

In [ ]:
aladin.overlay_survey="xcatdb/P/XMM/PN/color"

In [ ]:
aladin.overlay_survey_opacity = 0.8


#I cant see any display

## Vizier catalog for xmm log

# 
Now we make use of the "pyvo" library which allows us to access the virtual observatory services, such as TAP (Table Access Protocol) service provided by VizieR at CDS. This service allows for querying a wide range of astronomical data stored in VizieR.

Using this service, we can easily access the log of observations XMM-NEWTON & Fermi.  This allows us to obtain important information that can find specific observations that match certain criteria, such as time of observation, celestial coordinates, target name, or observation ID. Such observation logs often cantain information about the data quality, data availability, cross-matching catalogues and much more.


In [ ]:
# give the address of the service, you can also directly visit the website
tap_vizier = pyvo.dal.TAPService("https://tapvizier.cds.unistra.fr/TAPVizieR/tap")

In [ ]:
# access to vizier metadata
result = tap_vizier.search("select * from TAP_SCHEMA.tables where schema_name = 'metaviz'").to_table()
result.pprint_all()

In [ ]:
metacatcols = tap_vizier.search("select * from columns where table_name = 'METAcat'").to_table()
metacatcols

In [ ]:
# catalogs are in metacat
xmm_DR13 = tap_vizier.search("""select top 1 * from METAcat
                  where title like '%XMM%' and title like'%DR13%'
                  """).to_table()
xmm_DR13

In [ ]:
#Vizier.find_catalogs(["xmm", "log"])

In [ ]:
# tables are in metatab
xmm_DR13_tables = tap_vizier.search(
    """select * from METAtab
    where catid = 9069
    """
).to_table()
xmm_DR13_tables

In [ ]:
# retrieve the whole xmmlog table
xmmlog = tap_vizier.search('select * from "IX/69/summary"').to_table()
xmmlog


In [ ]:
len(xmmlog)

In [ ]:
for column in xmmlog.colnames:
    print(column, ":", xmmlog[column].description)

In [ ]:
aladin.add_table(xmmlog, color="lightpink")

# I get an error - TypeError: add_table() got an unexpected keyword argument 'color'

Now chose a source and use the simbad tool

## Now build the STMOC

# 
STMOCs are useful for identifying temporal and spatial overlaps between different astronomical catalogs, which is particularly useful for studying GRBs. Since GRBs are transient events that can be observed across multiple wavelengths.

Using STMOCs can:
- Help in identifying overlaps between different observations from different instruments which can aid in confirming GRB detections.
- Help in obtaing information from different instruments and to perform various physical studies such as GRB light curves, spectra, and afterglows, leading to better modeling of the underlying physical processes.
- Help in pinpointing the exact sky regions where GRBs were observed, aiding in follow-up observations with other telescopes.
- Help in cross-matching process that can lead to statiscal studies of GRB populations.


#
To proceed we can use tap service to understand the structure and content of time-related data in the catalog.

In [ ]:
# information on time is in metatime
time_columns = tap_vizier.search("select * from columns where table_name='METAtime'")
time_columns

In [ ]:
# lets get metatime for the second table (catid=2) of the catalog (catid=9096)
xmm_DR13_tables = tap_vizier.search(
    """select * from METAtime JOIN METAcol
    on METAtime.catid = METAcol.catid
    and METAcol.colid = METAtime.colid
    and METAcol.tabid = METAtime.tabid
    where METAcol.catid = 9069 and METAcol.tabid = 2
    """
).to_table()
xmm_DR13_tables

# 
STMOCs can handle only specific format of time. Hence, we convert Modified Julian Date (MJD) columns to Astropy Time objects for easier manipulation and consistency.

In [ ]:
# TODO here check the time scale in the catalog ('tai', 'tcb', 'tcg', 'tdb', 'tt', 'ut1', 'utc', 'local')
# got help from Laurent https://xmm-tools.cosmos.esa.int/external/xmm_user_support/documentation/uhb/reftime.html
xmmlog["start_obs"] = Time(xmmlog["MJD0"], format='mjd', scale="tt")
xmmlog["end_obs"] = Time(xmmlog["MJD1"], format='mjd', scale="tt")
xmmlog

# 
To create STMOCs, it is very important to set the temporal and spatial resolution. 
We have chosen 0.5 days to account for daily varations in the GRB (???)
We have chosen 1 degree as its adequate resoltion for identifying a GRB (???)

In [ ]:
TimeMOC.time_resolution_to_order(TimeDelta(0.5, format="jd")) # 0.5 days is order 26

In [ ]:
MOC.spatial_resolution_to_order(Angle("1d")) # 1 degrees is order 6

In [ ]:
stmoc_xmm = STMOC.from_time_ranges_positions(
    times_start=xmmlog["start_obs"],
    times_end=xmmlog["end_obs"],
    lon = Longitude(xmmlog["RAJ2000"]),
    lat = Latitude(xmmlog["DEJ2000"]),
    time_depth=26, spatial_depth=6
)

# 
Shown below are a series of plots that visualize the STMOCs maps of XMM-Newton observations for different time ranges. The plots show how the patial coverage of XMM-Newton observations changes over time. This is important for understanding the temporal distribution of GRB observations. Such plots can be useful in identifying overlapping observation periods with other instruments.

In [ ]:
def add_to_plot(fig, label, wcs, title, moc):
    """Add a MOC to a plot."""
    ax = fig.add_subplot(label, projection=wcs)

    ax.grid(color="black", linestyle="dotted")
    ax.set_title(title)
    ax.set_xlabel("lon")
    ax.set_ylabel("lat")

    moc.fill(ax=ax, wcs=wcs, alpha=0.9, fill=True, linewidth=0, color="#00bb00")
    # moc.border(ax=ax, wcs=wcs, linewidth=1, color="green")


fig = plt.figure(figsize=(10, 10))

time_ranges = Time(
    [
        [["2000-01-01", "2005-01-01"]],
        [["2005-01-01", "2010-01-01"]],
        [["2010-01-01", "2015-01-01"]],
        [["2015-01-01", "2020-01-01"]],
    ],
    format="iso",
    scale="tdb",
    out_subfmt="date",
)
with WCS(
    fig,
    fov=330 * u.deg,
    center=SkyCoord(0, 0, unit="deg", frame="galactic"),
    coordsys="galactic",
    rotation=Angle(0, u.degree),
    projection="AIT",
) as wcs:
    for i in range(0, 4):
        tmoc = TimeMOC.from_time_ranges(
            min_times=time_ranges[i][0, 0],
            max_times=time_ranges[i][0, 1],
            delta_t=TimeDelta(0.5, scale="tdb", format="jd"),
        )

        moc_2mass = stmoc_xmm.query_by_time(tmoc)
        title = "XMM observations between \n{} and {}".format(
            time_ranges[i][0, 0].iso,
            time_ranges[i][0, 1].iso,
        )
        id_subplot = int("22" + str(i + 1))
        add_to_plot(fig, id_subplot, wcs, title, moc_2mass)

plt.show()

## Now fermi grb

#
The same process can be repeated for Fermi 

In [ ]:
fermi_grb_metacat = tap_vizier.search("""select * from METAcat
                              where title like '%Fermi%GRB%'
                              """).to_table()
fermi_grb_metacat


In [ ]:
fermi_grb_catid = 18930046

In [ ]:
fermi_grb_tables = tap_vizier.search(f"""select * from METAtab where catid={fermi_grb_catid}
""").to_table()
fermi_grb_tables

In [ ]:
fermi_grb_triggers_columns = tap_vizier.search("""select * from columns where table_name='J/ApJ/893/46/table4'""").to_table()
fermi_grb_triggers_columns

In [ ]:
fermi_grb_durations_columns = tap_vizier.search("""select * from columns where table_name='J/ApJ/893/46/table5'""").to_table()
fermi_grb_durations_columns

In [ ]:
# lets get metatime for the trigger table (tabid=1)
fermi_triggers_time_info = tap_vizier.search(
    f"""select * from METAtime JOIN METAcol
    on METAtime.catid = METAcol.catid
    and METAcol.colid = METAtime.colid
    and METAcol.tabid = METAtime.tabid
    where METAcol.catid = {fermi_grb_catid} and METAcol.tabid = 1
    """
).to_table()
fermi_triggers_time_info


In [ ]:
# let's query the joined tables (to get event start time and duration)
fermi = tap_vizier.search(
    """select * from "J/ApJ/893/46/table5"
    join "J/ApJ/893/46/table4" USING(Fermi)
    """
).to_table()
fermi

In [ ]:
# we have to define a specific time for vizier 

from astropy.time.formats import erfa, TimeFromEpoch
class TimeVizier(TimeFromEpoch):
    name = "viztime"
    unit = 1.0 / erfa.DAYSEC  # in days (1 day == 86400 seconds)
    epoch_val = "2000-01-01 00:00:00"
    epoch_val2 = None
    epoch_scale = "utc"
    epoch_format = "iso"

In [ ]:
fermi["start_time"] = Time(fermi["Obs"] - erfa.DAYSEC / 2., format="viztime", scale="utc")
fermi["end_time"] = Time(fermi["Obs"] + erfa.DAYSEC / 2., format="viztime", scale="utc")
fermi

In [ ]:
stmoc_fermi = STMOC.from_time_ranges_positions(
    times_start=fermi["start_time"],
    times_end=fermi["end_time"],
    lon = Longitude(fermi["RAJ2000"]),
    lat = Latitude(fermi["DEJ2000"]),
    time_depth=26, spatial_depth=5  # ask Ada why 5 here
)

# 
Now using the STMOC information for both XMM and Fermi, we can use intersection method to identify the common space-time regions covered by both Fermi and XMM-Newton observations. Additionally, all the events, that have been identified, are confirmed by two different telescopes, therefore we can know for certain that the GRBs identified are genuine. These give as output the time when both the telescopes with pointing at the same area of the sky. These come in handy specially for identifing simultaneous observations, which are crucial for multi-wavelength studies of transient events like Gamma-Ray Bursts (GRBs).

In [ ]:
intersection = stmoc_fermi.intersection(stmoc_xmm)
intersection

# 
By now we have seen the power of using STMOCs to identifying transcients events such as GRBs. This method can also be applied for 
different astrophysical objects such as supernova remnants which emit at different wavelengths. STMOCs is a powerful tool which
facilitates the identification of temporal and spatial overlaps, enabling comprehensive multi-wavelength analyses and efficient data management, ultimately advancing our understanding of the object
of interest.

In [ ]:
hess = "J/A+A/612/A3"
super_nova_remnant = "RX J1713.7-3946"

In [ ]:
from astroquery.simbad import Simbad

In [ ]:
Simbad.query_object("RX J1713.7-3946")

In [ ]:
help(intersection.contains)

In [ ]:
only_space = MOC.from_string("6/37910 16282 9576 17030 26013")

In [ ]:
from astropy.coordinates import Longitude, Latitude

In [ ]:
only_space.contains_lonlat(lon=Longitude("17 12 27", unit="deg"), lat=Latitude("-39 41.2", unit="deg"))

In [ ]:
only_space.query_vizier_table("J/A+A/612/A3")

In [ ]:
fermi_grbs_intersection = only_space.query_vizier_table("J/ApJ/893/46")
fermi_grbs_intersection

In [ ]:
aladin.add_table(fermi_grbs_intersection, color="hotpink", shape="cross")

In [ ]:
simbad = Simbad()
simbad.add_votable_fields("flux")

In [ ]:
simbad_result = simbad.query_objects([f"Fermi {id}" for id in fermi_grbs_intersection["Fermi"]])
simbad_result

In [ ]:
only_space.query_vizier_table("IX/69")